## Imports

In [ ]:
# Import necessary libraries
import os
import re
import mne
import pandas as pd
import numpy as np
from bids import BIDSLayout
from collections import defaultdict
import json
import scipy as sp
bids_root = 'G:\\datasets\\'

# --- Frequency Bands Configuration ---
frequency_bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 12),
    'low_beta': (12, 20),
    'high_beta': (20, 30),
    'low_gamma': (30, 60),
    'high_gamma': (60, 150)
}

## Load Participant Info

In [10]:
participants_file = os.path.join(bids_root,'participants.tsv')

# load into dictionary
participant_info = {}
participants_df = pd.read_csv(participants_file, sep='\t')
for index, row in participants_df.iterrows():
    participant_info[row['participant_id']] = {'age': row['age'], 'sex': row['sex']}
print(participant_info)

# Save to a file
import pickle
with open('participant_info.pkl', 'wb') as file:  # 'wb' means write in binary mode
    pickle.dump(participant_info, file)


{'sub-ccepAgeUMCU01': {'age': 15, 'sex': 'male'}, 'sub-ccepAgeUMCU02': {'age': 35, 'sex': 'female'}, 'sub-ccepAgeUMCU03': {'age': 25, 'sex': 'male'}, 'sub-ccepAgeUMCU04': {'age': 4, 'sex': 'male'}, 'sub-ccepAgeUMCU05': {'age': 17, 'sex': 'female'}, 'sub-ccepAgeUMCU06': {'age': 26, 'sex': 'male'}, 'sub-ccepAgeUMCU07': {'age': 10, 'sex': 'female'}, 'sub-ccepAgeUMCU08': {'age': 9, 'sex': 'female'}, 'sub-ccepAgeUMCU09': {'age': 21, 'sex': 'male'}, 'sub-ccepAgeUMCU10': {'age': 15, 'sex': 'male'}, 'sub-ccepAgeUMCU11': {'age': 5, 'sex': 'male'}, 'sub-ccepAgeUMCU12': {'age': 14, 'sex': 'male'}, 'sub-ccepAgeUMCU13': {'age': 45, 'sex': 'male'}, 'sub-ccepAgeUMCU14': {'age': 10, 'sex': 'female'}, 'sub-ccepAgeUMCU15': {'age': 15, 'sex': 'male'}, 'sub-ccepAgeUMCU16': {'age': 42, 'sex': 'female'}, 'sub-ccepAgeUMCU17': {'age': 4, 'sex': 'male'}, 'sub-ccepAgeUMCU18': {'age': 15, 'sex': 'female'}, 'sub-ccepAgeUMCU19': {'age': 25, 'sex': 'female'}, 'sub-ccepAgeUMCU20': {'age': 11, 'sex': 'male'}, 'sub-cc

## Extract Data Pipeline

In [ ]:
def get_artifact_electrodes(ch_info,e1,e2,thresh_mm=13):
    e1_xyz = ch_info[e1]['xyz']
    e2_xyz = ch_info[e2]['xyz']
    artifact_electrodes = []
    for ch_key,ch_data in ch_info.items():
        e1_dist = np.linalg.norm(ch_data['xyz'] - e1_xyz)
        e2_dist = np.linalg.norm(ch_data['xyz'] - e2_xyz)
        if e1_dist <= thresh_mm or e2_dist <= thresh_mm:
            artifact_electrodes.append(ch_key.upper())
    return artifact_electrodes

def get_exyz(el_info_df,e_name):
    x = el_info_df[el_info_df['name'] == e_name]['x'].to_numpy()
    y = el_info_df[el_info_df['name'] == e_name]['y'].to_numpy()
    z = el_info_df[el_info_df['name'] == e_name]['z'].to_numpy()
    return np.array((x,y,z))[:,0]


for participant_id in participant_info.keys():

    subject_id = participant_id.split('-')[1]
    task_label = 'SPESclin'

    # --- Downsampling Configuration ---
    TARGET_SFREQ = 512.0 # Hz 
    # ----------------------------------

    # --- Filtering Configuration ---
    HIGH_PASS_FREQ = 1.0 # Hz
    LOW_PASS_FREQ = 150.0 # Hz 
    # -----------------------------

    # --- Event and Epoching Configuration ---
    STIMULATION_EVENT_COLUMN = 'trial_type' 
    STIMULATION_EVENT_VALUES = ['electrical_stimulation'] 

    # --- Channel Type Configuration (for selecting ECOG/SEEG) ---
    KEEP_CHANNEL_TYPES = ['ECOG', 'SEEG']

    # --- Updated Epoching Parameters based on Paper (s41593-023-01272-0) ---
    BASELINE_CORRECTION = None 
    PAPER_BASELINE_WINDOW = (-1.0, -0.1) 

    STIM_CHANNEL_COLS = ['electrical_stimulation_site'] # MODIFY THIS BASED ON YOUR events.tsv FILE!

    # --- Epoching Parameters ---
    INITIAL_EPOCH_TMIN = -2.0
    INITIAL_EPOCH_TMAX = 3.0
    PAPER_BASELINE_WINDOW = (-1.0, -0.1) 
    FINAL_EPOCH_TMIN = 0.009 # 9 ms post-stimulation
    FINAL_EPOCH_TMAX = 1.0   # 1 s post-stimulation
    # -------------------------------------------------

    # Find iEEG data files for the subject and task, get BIDSFile objects
    layout = BIDSLayout(bids_root, validate=True, index_metadata=False)
    ieeg_bids_files = layout.get(
        subject=subject_id,
        task=task_label,
        suffix='ieeg',
        extension=['.vhdr','.edf','.bdf'], # Add other relevant extensions if needed
        return_type='object'
    )

    # %% Check if files were found
    if not ieeg_bids_files:
        print(f"No iEEG data files found for subject {subject_id}, task {task_label}.")
        exit()

    print(f"Found {len(ieeg_bids_files)} potential iEEG recordings for {subject_id}, task {task_label}.")

    # %% Process each recording run
    participant_data_by_run = {}

    for ieeg_file in ieeg_bids_files:
        # Keep entities for finding channels.tsv later AND for storing
        run_key_entities = ieeg_file.entities.copy()
        run_id = ieeg_file.entities.get('run', 'run-unknown')
        session_id = ieeg_file.entities.get('session', 'ses-unknown')
        recording_key = f"{session_id}_{run_id}"

        print(f"\n--- Processing Recording: {recording_key} ---")
        print(f"iEEG file: {ieeg_file.path}")

        # Find the corresponding event file
        assoc_entities = run_key_entities.copy()
        assoc_entities.pop('extension', None)
        assoc_entities.pop('suffix', None)

        event_file_list = layout.get(
            return_type='object',
            suffix='events',
            extension='.tsv',
            **assoc_entities
        )

        event_file = event_file_list[0]
        print(f"Event file: {event_file.path}")

        # Find Channels file (needed early for dropping bad channels)
        channels_file_list = layout.get(return_type='filename', suffix='channels', extension='.tsv', **assoc_entities)
        channels_file = channels_file_list[0] if channels_file_list else None
        print(f"Channels file: {channels_file}")

        # Process Events and Epoch Data
        events_df = pd.read_csv(event_file.path, sep='\t')

        # --- Check for essential columns ---
        required_event_cols = [STIMULATION_EVENT_COLUMN, 'onset'] + STIM_CHANNEL_COLS
        missing_cols = [col for col in required_event_cols if col not in events_df.columns]

        # --- Filter for stimulation events ---
        stim_events_df = events_df[events_df[STIMULATION_EVENT_COLUMN].isin(STIMULATION_EVENT_VALUES)].copy()
        print(f"Found {len(stim_events_df)} stimulation events in {event_file.path}.")

        # --- Prepare Metadata for Epochs ---
        metadata_cols_to_keep = ['onset', 'duration'] + STIM_CHANNEL_COLS #  + [col for col in OTHER_METADATA_COLS if col in events_df.columns]
        # Ensure we only select columns that actually exist in stim_events_df
        metadata_cols_to_keep = [col for col in metadata_cols_to_keep if col in stim_events_df.columns]
        stim_metadata_df = stim_events_df[metadata_cols_to_keep].reset_index(drop=True)
        print(f"Prepared metadata with columns: {list(stim_metadata_df.columns)}")

        # Load the raw data into memory for resampling/filtering
        raw = mne.io.read_raw(ieeg_file.path, preload=True)
        original_sfreq = raw.info['sfreq']
        print(f"Raw data loaded into memory.")

        # --- Apply Filters ---
        if HIGH_PASS_FREQ is not None or LOW_PASS_FREQ is not None:
            filter_desc = []
            if HIGH_PASS_FREQ is not None: filter_desc.append(f"{HIGH_PASS_FREQ} Hz high-pass")
            if LOW_PASS_FREQ is not None: filter_desc.append(f"{LOW_PASS_FREQ} Hz low-pass")
            print(f"Applying {' and '.join(filter_desc)} filter...")
            raw.filter(l_freq=HIGH_PASS_FREQ, h_freq=LOW_PASS_FREQ,
                        picks='eeg', method='fir', phase='zero',
                        fir_window='hamming', verbose=False)

        # --- Downsample if necessary ---
        current_sfreq = original_sfreq
        if original_sfreq > TARGET_SFREQ:
            print(f"Original sfreq ({original_sfreq} Hz) > target ({TARGET_SFREQ} Hz). Downsampling...")
            raw.resample(sfreq=TARGET_SFREQ, npad='auto', verbose=False)
            current_sfreq = raw.info['sfreq']
            print(f"Downsampling complete. New sfreq: {current_sfreq} Hz")

        # --- Create MNE Events Array ---
        onsets_sec = stim_events_df['onset'].values
        onsets_sample = (onsets_sec * current_sfreq).astype(int)
        event_id_value = 1
        event_ids = np.full(len(onsets_sample), event_id_value)
        mne_events = np.vstack((onsets_sample, np.zeros_like(onsets_sample), event_ids)).T

        # --- Create Epochs with Metadata ---
        event_id_dict_mne = {STIMULATION_EVENT_VALUES[0]: event_id_value}
        print(f"Creating epochs around {len(mne_events)} events: tmin={INITIAL_EPOCH_TMIN}, tmax={INITIAL_EPOCH_TMAX}")
        epochs = mne.Epochs(
            raw=raw, # Use the potentially downsampled and filtered raw data
            events=mne_events,
            event_id=event_id_dict_mne,
            tmin=INITIAL_EPOCH_TMIN,
            tmax=INITIAL_EPOCH_TMAX,
            metadata=stim_metadata_df, 
            baseline=None, # Set to None
            preload=True, # Keep True as data is already loaded
            reject=None,
            reject_by_annotation=False,
            picks='eeg'
        )
        print(f"Successfully created {len(epochs)} epochs with metadata (Baseline not applied yet).")

        # --- Apply Custom Median Baseline Correction ---
        print(f"Applying MEDIAN baseline correction using window: {PAPER_BASELINE_WINDOW}s")
        baseline_epochs = epochs.copy().crop(tmin=PAPER_BASELINE_WINDOW[0], tmax=PAPER_BASELINE_WINDOW[1])
        baseline_data = baseline_epochs.get_data(copy=False) 
        baseline_medians = np.median(baseline_data, axis=2) 
        epoch_data = epochs.get_data(copy=False) 
        epoch_data_baselined = epoch_data - baseline_medians[:, :, np.newaxis]
        epochs_baselined = mne.EpochsArray(epoch_data_baselined, epochs.info,
                                            events=epochs.events, tmin=epochs.tmin,
                                            event_id=epochs.event_id, metadata=epochs.metadata)

        # --- Crop Epochs to Final Time Window ---
        print(f"Cropping epochs to final window: {FINAL_EPOCH_TMIN}s to {FINAL_EPOCH_TMAX}s")
        epochs_baselined.crop(tmin=FINAL_EPOCH_TMIN, tmax=FINAL_EPOCH_TMAX, include_tmax=True)


        # --- Identify Channels to Drop (based on Type and Explicit Excludes) ---
        epochs_baselined_all_channels = epochs_baselined.copy()
        channels_to_drop_by_type = []
        channels_df = pd.read_csv(channels_file, sep='\t')
        keep_types_lower = [t.lower() for t in KEEP_CHANNEL_TYPES]
        channels_df['type_lower'] = channels_df['type'].astype(str).str.lower()
        channels_to_drop_by_type = channels_df[~channels_df['type_lower'].isin(keep_types_lower)]['name'].tolist()
        print(f"Identified {len(channels_to_drop_by_type)} channels not matching types {KEEP_CHANNEL_TYPES} in {channels_file}.")
        bad_channels_list = []
        channels_df = pd.read_csv(channels_file, sep='\t')
        bad_channels_list = channels_df[~channels_df['status'].isin(['good'])]['name'].tolist()
        combined_channels_to_drop = list(set(bad_channels_list + channels_to_drop_by_type))
        channels_present_to_drop = [ch for ch in combined_channels_to_drop if ch in epochs_baselined.ch_names]
        if channels_present_to_drop:
                print(f"Dropping channels: {channels_present_to_drop}")
                epochs_baselined.drop_channels(channels_present_to_drop)
                print(f"Channels dropped. New n_channels: {len(epochs_baselined.ch_names)}")
        else:
                print("No channels identified to drop based on type or exclusion list.")

        # Store data for this run - ONLY if successful
        participant_data_by_run[recording_key] = {
            'epochs': epochs_baselined, # Store final epochs object (now includes metadata)
            'epochs_all_chan': epochs_baselined_all_channels, # Store final epochs object (now includes metadata)
            'ieeg_file_path': ieeg_file.path,
            'event_file_path': event_file.path,
            'sfreq': current_sfreq,
            'run_entities' : run_key_entities
        }
        del raw

    # --- Concatenate Epochs & Extract Metadata ---
    final_stim_data = {}

    # Group epochs by unique channel sets AND FINAL sampling frequency
    epochs_by_config = defaultdict(list)
    run_keys_by_config = defaultdict(list)
    for run_key, run_data in participant_data_by_run.items():
        epochs_obj = run_data['epochs']
        sfreq = run_data['sfreq']
        ch_names_tuple = tuple(epochs_obj.ch_names)
        config_key = (ch_names_tuple, sfreq)
        epochs_by_config[config_key].append(epochs_obj)
        run_keys_by_config[config_key].append(run_key)
    print(f"\nFound {len(epochs_by_config)} unique configuration(s) (good channels AND final sfreq).")

    for run_key, run_data in participant_data_by_run.items():
        (ch_names_tuple, sfreq) = (tuple(run_data['epochs'].ch_names),run_data['sfreq'])
        num_channels = len(ch_names_tuple)
        pseudo_subject_key = f"{subject_id}_{run_key}"
        print(f"\nProcessing run {run_key}")
        print(f"  Pseudo-subject key: {pseudo_subject_key}")

        # --- Get Representative Entities for Metadata Lookup (No Run) ---
        metadata_lookup_entities = {} # Prepare dict for metadata lookup
        metadata_lookup_entities = run_data['run_entities'].copy()
        metadata_lookup_entities.pop('run', None) # Remove run
        metadata_lookup_entities.pop('task', None) # Remove task
        metadata_lookup_entities.pop('extension', None)
        metadata_lookup_entities.pop('suffix', None)
        print(f"  Using entities for metadata lookup: {metadata_lookup_entities}")

        # --- Extract SOZ Info ---
        soz_areas = []
        resected_areas = []
        electrodes_file_list = layout.get(return_type='filename', suffix='electrodes', extension='.tsv', **metadata_lookup_entities)
        electrodes_file = electrodes_file_list[0]
        electrodes_df = pd.read_csv(electrodes_file, sep='\t')
        for _, row in electrodes_df.iterrows():
            if row['soz'] == 'yes':
                soz_areas.append(row['Destrieux_label'])
            if row['resected'] == 'yes':
                resected_areas.append(row['Destrieux_label'])
        soz_array = np.zeros(num_channels, dtype=int)
        resected_array = np.zeros(num_channels,dtype=int)
        soz_map = {row['name']: 1 if row['Destrieux_label'] in soz_areas else 0 for _, row in electrodes_df.iterrows()}
        resected_map = {row['name']: 1 if row['Destrieux_label'] in resected_areas else 0 for _, row in electrodes_df.iterrows()}
        current_ch_names = list(ch_names_tuple)
        for i, ch_name in enumerate(current_ch_names): 
            soz_array[i] = soz_map.get(ch_name,0)
            resected_array[i] = resected_map.get(ch_name,0)
        print(f"  Stored SOZ info array (shape: {soz_array.shape}, sum: {np.sum(soz_array)}).")

        # --- Extract Electrode Information ---
        ch_info_dict = {}
        current_ch_names = list(ch_names_tuple)
        for ch_i, ch_data in electrodes_df.iterrows():
            if ch_data['name'] not in current_ch_names:
                continue
            ch_dict = {}
            ch_dict['index'] = ch_i #ch_data['index']
            ch_dict['xyz'] = np.array((ch_data['x'],ch_data['y'],ch_data['z']))
            ch_dict['Destrieux_label'] = ch_data['Destrieux_label']
            ch_dict['Destrieux_label_text'] = ch_data['Destrieux_label_text']
            ch_info_dict[ch_data['name'].upper()] = ch_dict
        all_ch_info_dict = {}
        for ch_i, ch_data in electrodes_df.iterrows():
            ch_dict = {}
            ch_dict['index'] = ch_i #ch_data['index']
            ch_dict['xyz'] = np.array((ch_data['x'],ch_data['y'],ch_data['z']))
            ch_dict['Destrieux_label'] = ch_data['Destrieux_label']
            ch_dict['Destrieux_label_text'] = ch_data['Destrieux_label_text']
            all_ch_info_dict[ch_data['name'].upper()] = ch_dict

        # --- Extract Stim Info ---
        events_lookup_entities = {} 
        events_lookup_entities = run_data['run_entities'].copy()
        events_lookup_entities.pop('task', None) 
        events_lookup_entities.pop('extension', None)
        events_lookup_entities.pop('suffix', None)
        epoch_onset_times = run_data['epochs'].events[:,0]
        config_stim_info = {}
        stim_event_file = layout.get(return_type='filename', suffix='events', extension='.tsv', **events_lookup_entities)[0]
        stim_df = pd.read_csv(stim_event_file, sep='\t')
        for iRow,row in stim_df.iterrows():
            onset_sample = int(row['onset']*run_data['epochs'].info['sfreq'])
            if onset_sample not in epoch_onset_times:
                continue
            stim_trial = {}
            stim_trial['stim_electrodes'] = row['electrical_stimulation_site']
            anode = stim_trial['stim_electrodes'].split('-')[0]
            cathode = stim_trial['stim_electrodes'].split('-')[1]
            stim_trial['stim_site'] = (anode,cathode)
            stim_trial['stim_labels'] = (all_ch_info_dict[anode.upper()]['Destrieux_label'],all_ch_info_dict[cathode.upper()]['Destrieux_label'])
            stim_trial['stim_labels_text'] = (all_ch_info_dict[anode.upper()]['Destrieux_label_text'],all_ch_info_dict[cathode.upper()]['Destrieux_label_text'])
            stim_trial['stim_xyz'] = (all_ch_info_dict[anode.upper()]['xyz'],all_ch_info_dict[cathode.upper()]['xyz'])
            stim_trial['stim_type'] = row['electrical_stimulation_type']
            stim_trial['stim_current'] = row['electrical_stimulation_current']
            stim_trial['stim_frequency'] = row['electrical_stimulation_frequency']
            stim_trial['stim_pulsewidth'] = row['electrical_stimulation_pulsewidth']
            config_stim_info[iRow] = stim_trial

        # --- Extract Subject Info ---
        subject_info = {}
        subject_id = pseudo_subject_key.split('_')[0]
        subject_info['subject_id'] = subject_id
        subject_info['age'] = participant_info['sub-' + subject_id]['age']
        subject_info['sex'] = participant_info['sub-' + subject_id]['sex']

        # --- Remove artifact electrodes from timeseries data ---
        epoch_data = run_data['epochs'].get_data(copy=True) # Shape: (n_epochs, n_channels, n_times)
        epoch_ch_names = run_data['epochs'].ch_names
        epoch_ch_names = [epoch_ch_name.upper() for epoch_ch_name in epoch_ch_names]
        epoch_data_all_chan = run_data['epochs_all_chan'].get_data(copy=True)
        epoch_data_all_ch_names = run_data['epochs_all_chan'].ch_names
        epoch_data_all_ch_names = [epoch_ch_name.upper() for epoch_ch_name in epoch_data_all_ch_names]

        cleaned_stim_data = {}
        iTrial = 0
        for _,trial_stim_data in config_stim_info.items():
            anode,cathode = trial_stim_data['stim_site']
            artifact_electrodes = get_artifact_electrodes(all_ch_info_dict,anode.upper(),cathode.upper())
            artifact_indices = [epoch_ch_names.index(artifact_electrode) for artifact_electrode in artifact_electrodes if artifact_electrode in epoch_ch_names]
            artifact_indices = np.array(artifact_indices).astype(int)
            artifact_mask = np.ones(epoch_data[iTrial].shape[0],bool)
            artifact_mask[artifact_indices] = False

            stim_indices = np.array([epoch_data_all_ch_names.index(anode.upper()),epoch_data_all_ch_names.index(cathode.upper())]).astype(int)
            stim_mask = np.zeros(epoch_data_all_chan[iTrial].shape[0],bool)
            stim_mask[stim_indices] = True
            cleaned_ecog_data = epoch_data[iTrial,artifact_mask,:]

            cleaned_ch_info = ch_info_dict.copy()
            for artifact_electrode in artifact_electrodes:
                if artifact_electrode in cleaned_ch_info: 
                    del cleaned_ch_info[artifact_electrode]

            cleaned_epoch_data = {}
            cleaned_epoch_data['ecog_data'] = cleaned_ecog_data
            cleaned_epoch_data['stim_data'] = epoch_data_all_chan[iTrial,stim_mask,:]
            cleaned_epoch_data['soz_labels'] = soz_array[artifact_mask]
            cleaned_epoch_data['res_labels'] = resected_array[artifact_mask]
            cleaned_epoch_data['ch_info'] = cleaned_ch_info
            cleaned_epoch_data['stim_info'] = trial_stim_data
            cleaned_epoch_data['subject_info'] = subject_info
            cleaned_stim_data[iTrial] = cleaned_epoch_data
            iTrial += 1
        final_stim_data[pseudo_subject_key] = cleaned_stim_data

    for config_key,run_stim_data in final_stim_data.items():
        # Save to a file
        import pickle
        with open('.\\processed_data\\' + config_key + '.pkl', 'wb') as file: 
            pickle.dump(run_stim_data, file)


c:\Users\tyler_p5hze3w\anaconda3\envs\park_dynamics\Lib\site-packages\bids\layout\validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "


Found 8 potential iEEG recordings for ccepAgeUMCU01, task SPESclin.

--- Processing Recording: 1_021448 ---
iEEG file: G:\datasets\sub-ccepAgeUMCU01\ses-1\ieeg\sub-ccepAgeUMCU01_ses-1_task-SPESclin_run-021448_ieeg.vhdr
Event file: G:\datasets\sub-ccepAgeUMCU01\ses-1\ieeg\sub-ccepAgeUMCU01_ses-1_task-SPESclin_run-021448_events.tsv
Channels file: G:\datasets\sub-ccepAgeUMCU01\ses-1\ieeg\sub-ccepAgeUMCU01_ses-1_task-SPESclin_run-021448_channels.tsv
Found 460 stimulation events in G:\datasets\sub-ccepAgeUMCU01\ses-1\ieeg\sub-ccepAgeUMCU01_ses-1_task-SPESclin_run-021448_events.tsv.
Prepared metadata with columns: ['onset', 'duration', 'electrical_stimulation_site']
Extracting parameters from G:\datasets\sub-ccepAgeUMCU01\ses-1\ieeg\sub-ccepAgeUMCU01_ses-1_task-SPESclin_run-021448_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 3686479  =      0.000 ...  7200.154 secs...
Raw data loaded into memory.
Applying 1.0 Hz high-pass and 150.0 Hz low-pass filter...
Creating epochs around

c:\Users\tyler_p5hze3w\anaconda3\envs\park_dynamics\Lib\site-packages\bids\layout\validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "


Found 3 potential iEEG recordings for ccepAgeUMCU02, task SPESclin.

--- Processing Recording: 1_021804 ---
iEEG file: G:\datasets\sub-ccepAgeUMCU02\ses-1\ieeg\sub-ccepAgeUMCU02_ses-1_task-SPESclin_run-021804_ieeg.vhdr
Event file: G:\datasets\sub-ccepAgeUMCU02\ses-1\ieeg\sub-ccepAgeUMCU02_ses-1_task-SPESclin_run-021804_events.tsv
Channels file: G:\datasets\sub-ccepAgeUMCU02\ses-1\ieeg\sub-ccepAgeUMCU02_ses-1_task-SPESclin_run-021804_channels.tsv
Found 508 stimulation events in G:\datasets\sub-ccepAgeUMCU02\ses-1\ieeg\sub-ccepAgeUMCU02_ses-1_task-SPESclin_run-021804_events.tsv.
Prepared metadata with columns: ['onset', 'duration', 'electrical_stimulation_site']
Extracting parameters from G:\datasets\sub-ccepAgeUMCU02\ses-1\ieeg\sub-ccepAgeUMCU02_ses-1_task-SPESclin_run-021804_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 7062911  =      0.000 ...  3448.687 secs...
Raw data loaded into memory.
Applying 1.0 Hz high-pass and 150.0 Hz low-pass filter...
Original sfreq (2048.0

c:\Users\tyler_p5hze3w\anaconda3\envs\park_dynamics\Lib\site-packages\bids\layout\validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "


...

Found 1 potential iEEG recordings for ccepAgeUMCU72, task SPESclin.

--- Processing Recording: 1_021054 ---
iEEG file: G:\datasets\sub-ccepAgeUMCU72\ses-1\ieeg\sub-ccepAgeUMCU72_ses-1_task-SPESclin_run-021054_ieeg.vhdr
Event file: G:\datasets\sub-ccepAgeUMCU72\ses-1\ieeg\sub-ccepAgeUMCU72_ses-1_task-SPESclin_run-021054_events.tsv
Channels file: G:\datasets\sub-ccepAgeUMCU72\ses-1\ieeg\sub-ccepAgeUMCU72_ses-1_task-SPESclin_run-021054_channels.tsv
Found 505 stimulation events in G:\datasets\sub-ccepAgeUMCU72\ses-1\ieeg\sub-ccepAgeUMCU72_ses-1_task-SPESclin_run-021054_events.tsv.
Prepared metadata with columns: ['onset', 'duration', 'electrical_stimulation_site']
Extracting parameters from G:\datasets\sub-ccepAgeUMCU72\ses-1\ieeg\sub-ccepAgeUMCU72_ses-1_task-SPESclin_run-021054_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 6217279  =      0.000 ...  3035.780 secs...
Raw data loaded into memory.
Applying 1.0 Hz high-pass and 150.0 Hz low-pass filter...
Original sfreq (2048.0

c:\Users\tyler_p5hze3w\anaconda3\envs\park_dynamics\Lib\site-packages\bids\layout\validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "


Found 3 potential iEEG recordings for ccepAgeUMCU73, task SPESclin.

--- Processing Recording: 1_021140 ---
iEEG file: G:\datasets\sub-ccepAgeUMCU73\ses-1\ieeg\sub-ccepAgeUMCU73_ses-1_task-SPESclin_run-021140_ieeg.vhdr
Event file: G:\datasets\sub-ccepAgeUMCU73\ses-1\ieeg\sub-ccepAgeUMCU73_ses-1_task-SPESclin_run-021140_events.tsv
Channels file: G:\datasets\sub-ccepAgeUMCU73\ses-1\ieeg\sub-ccepAgeUMCU73_ses-1_task-SPESclin_run-021140_channels.tsv
Found 180 stimulation events in G:\datasets\sub-ccepAgeUMCU73\ses-1\ieeg\sub-ccepAgeUMCU73_ses-1_task-SPESclin_run-021140_events.tsv.
Prepared metadata with columns: ['onset', 'duration', 'electrical_stimulation_site']
Extracting parameters from G:\datasets\sub-ccepAgeUMCU73\ses-1\ieeg\sub-ccepAgeUMCU73_ses-1_task-SPESclin_run-021140_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 5179775  =      0.000 ...  2529.187 secs...
Raw data loaded into memory.
Applying 1.0 Hz high-pass and 150.0 Hz low-pass filter...
Original sfreq (2048.0

c:\Users\tyler_p5hze3w\anaconda3\envs\park_dynamics\Lib\site-packages\bids\layout\validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "


Found 1 potential iEEG recordings for ccepAgeUMCU74, task SPESclin.

--- Processing Recording: 1_041028 ---
iEEG file: G:\datasets\sub-ccepAgeUMCU74\ses-1\ieeg\sub-ccepAgeUMCU74_ses-1_task-SPESclin_run-041028_ieeg.vhdr
Event file: G:\datasets\sub-ccepAgeUMCU74\ses-1\ieeg\sub-ccepAgeUMCU74_ses-1_task-SPESclin_run-041028_events.tsv
Channels file: G:\datasets\sub-ccepAgeUMCU74\ses-1\ieeg\sub-ccepAgeUMCU74_ses-1_task-SPESclin_run-041028_channels.tsv
Found 550 stimulation events in G:\datasets\sub-ccepAgeUMCU74\ses-1\ieeg\sub-ccepAgeUMCU74_ses-1_task-SPESclin_run-041028_events.tsv.
Prepared metadata with columns: ['onset', 'duration', 'electrical_stimulation_site']
Extracting parameters from G:\datasets\sub-ccepAgeUMCU74\ses-1\ieeg\sub-ccepAgeUMCU74_ses-1_task-SPESclin_run-041028_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 7372991  =      0.000 ...  3600.093 secs...
Raw data loaded into memory.
Applying 1.0 Hz high-pass and 150.0 Hz low-pass filter...
Original sfreq (2048.0

## Extract SOZ INFO

In [ ]:
from collections import Counter, defaultdict

# Define the mapping of fine-grained labels to coarse categories
coarse_map = {
    'Pole_occipital': 'Occipital',
    'G_occipital_middle': 'Occipital',
    'G_cuneus': 'Occipital',
    'S_calcarine': 'Occipital',
    'G&S_occipital_inf': 'Occipital',
    'S_occipital_ant': 'Occipital',
    
    'G_temporal_middle': 'Temporal',
    'G_temporal_inf': 'Temporal',
    'Pole_temporal': 'Temporal',
    'G_temp_sup-Lateral': 'Temporal',
    'G_temp_sup-Plan_tempo': 'Temporal',
    'G_temp_sup-Plan_polar': 'Temporal',
    'S_temporal_inf': 'Temporal',
    'S_temporal_sup': 'Temporal',
    'G_oc-temp_lat-fusifor': 'Temporal',
    'G_oc-temp_med-Parahip': 'Temporal',
    'G_oc-temp_med-Lingual': 'Temporal',
    
    'G_pariet_inf-Supramar': 'Parietal',
    'G_pariet_inf-Angular': 'Parietal',
    'G_parietal_sup': 'Parietal',
    'G_postcentral': 'Parietal',
    'S_postcentral': 'Parietal',
    'S_intrapariet&P_trans': 'Parietal',
    
    'G_front_sup': 'Frontal',
    'G_front_middle': 'Frontal',
    'G_front_inf-Orbital': 'Frontal',
    'G_front_inf-Opercular': 'Frontal',
    'S_front_sup': 'Frontal',
    'S_front_middle': 'Frontal',
    'S_front_inf': 'Frontal',
    'G_precentral': 'Frontal',
    'S_precentral-sup-part': 'Frontal',
    'S_precentral-inf-part': 'Frontal',
    'G&S_paracentral': 'Frontal',
    'G&S_subcentral': 'Frontal',
    'G_rectus': 'Frontal',
    'G_orbital': 'Frontal',
    'S_orbital-H_Shaped': 'Frontal',
    
    'G&S_cingul-Mid-Post': 'Limbic',
    'G_cingul-Post-dorsal': 'Limbic',
    'G&S_cingul-Ant': 'Limbic',
    'G_insular_short': 'Limbic',
    'S_circular_insula_sup': 'Limbic',
    'S_circular_insula_ant': 'Limbic',
    
    'G_precuneus': 'Precuneus',
    'S_subparietal': 'Precuneus',
    
    'Lat_Fis-post': 'Other',
    'G_temp_sup-G_T_transv': 'Other',
    'S_collat_transv_ant': 'Other',
    'Unknown': 'Other',
}

# Find iEEG data files for the subject and task, get BIDSFile objects
layout = BIDSLayout(bids_root,validate=True,index_metadata=False)
ieeg_bids_files = layout.get(
    subject='ccepAgeUMCU01',
    task=task_label,
    suffix='ieeg',
    extension=['.vhdr','.edf','.bdf'], # Add other relevant extensions if needed
    return_type='object'
)


# --- Extract SOZ Info ---
soz_areas = []
resected_areas = []
electrodes_file_list = layout.get(return_type='filename', suffix='electrodes', extension='.tsv')
print(electrodes_file_list)

all_soz_labels = {}
all_soz_centers = {}
all_res_labels = {}
for participant_id in participant_info.keys():

    subject_id = participant_id.split('-')[1]
    subject_id_int = int(re.search(r'\d+', subject_id).group())
    

    # Define the task label based on previous information
    task_label = 'SPESclin'
    electrodes_file = [filename for filename in electrodes_file_list if subject_id in filename]
    electrodes_df = pd.read_csv(electrodes_file[0], sep='\t')

    # print(electrodes_file)
    soz_areas = []
    soz_xyz = []
    res_areas = []
    for _, row in electrodes_df.iterrows():
        if row['soz'] == 'yes':
            soz_areas.append((row['Destrieux_label'],row['Destrieux_label_text']))
            soz_xyz.append(np.array([row['x'],row['y'],row['z']]))
        if row['resected'] == 'yes':
            res_areas.append((row['Destrieux_label'],row['Destrieux_label_text']))

    if len(soz_areas) > 0:

        coarse_labels = []
        for _,label in soz_areas:
            coarse = coarse_map.get(label,'Other')
            coarse_labels.append(coarse)

        count = Counter(coarse_labels)
        dominant = count.most_common(1)[0][0]  

        all_soz_labels[subject_id_int] = dominant
        soz_mean =  np.median(np.array(soz_xyz),axis=0)
        print(soz_mean)
        all_soz_centers[subject_id_int] = soz_mean

    if len(res_areas) > 0:

        coarse_labels = []
        for _,label in res_areas:
            coarse = coarse_map.get(label,'Other')
            coarse_labels.append(coarse)

        count = Counter(coarse_labels)
        dominant = count.most_common(1)[0][0]  
        all_res_labels[subject_id_int] = dominant


print(len(all_soz_labels),all_soz_labels)
print(len(all_res_labels),all_res_labels)

import pickle
with open('.\\soz_cat_labels.pkl', 'wb') as file: 
    pickle.dump(all_soz_labels, file)

with open('.\\res_cat_labels.pkl', 'wb') as file: 
    pickle.dump(all_res_labels, file)

with open('.\\soz_xyz_centers.pkl', 'wb') as file: 
    pickle.dump(all_soz_centers, file)


c:\Users\tyler_p5hze3w\anaconda3\envs\park_dynamics\Lib\site-packages\bids\layout\validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "


['G:\\datasets\\sub-ccepAgeUMCU01\\ses-1\\ieeg\\sub-ccepAgeUMCU01_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU02\\ses-1\\ieeg\\sub-ccepAgeUMCU02_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU03\\ses-1\\ieeg\\sub-ccepAgeUMCU03_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU04\\ses-1\\ieeg\\sub-ccepAgeUMCU04_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU05\\ses-1\\ieeg\\sub-ccepAgeUMCU05_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU06\\ses-1\\ieeg\\sub-ccepAgeUMCU06_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU07\\ses-1\\ieeg\\sub-ccepAgeUMCU07_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU08\\ses-1\\ieeg\\sub-ccepAgeUMCU08_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU09\\ses-1b\\ieeg\\sub-ccepAgeUMCU09_ses-1b_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU10\\ses-1\\ieeg\\sub-ccepAgeUMCU10_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMCU11\\ses-1\\ieeg\\sub-ccepAgeUMCU11_ses-1_electrodes.tsv', 'G:\\datasets\\sub-ccepAgeUMC

In [ ]:

all_soz_labels_comb = {}
all_res_labels_comb = {}
tie_threshold = 2 # Consider counts close if difference is <= tie_threshold

for participant_id in participant_info.keys():
    subject_id_bids = participant_id

    subject_id_int = int(re.search(r'\d+', subject_id_bids).group())

    electrodes_file = [filename for filename in electrodes_file_list if subject_id_bids in filename]
    electrodes_df = pd.read_csv(electrodes_file[0], sep='\t')
    print(participant_id,electrodes_file)

    # --- Process SOZ ---
    soz_df = electrodes_df[electrodes_df['soz'] == 'yes']

    if not soz_df.empty:
        coarse_labels_soz = [coarse_map.get(label, 'Other') for label in soz_df['Destrieux_label_text']]
        
        if not coarse_labels_soz:
             all_soz_labels_comb[subject_id_int] = 'Unknown'
             continue

        count_soz = Counter(coarse_labels_soz)
        top_two = count_soz.most_common(2)
        
        if len(top_two) == 1: # Only one category present
            all_soz_labels_comb[subject_id_int] = top_two[0][0]
        elif len(top_two) == 2:
            # Check if counts are equal or close
            if abs(top_two[0][1] - top_two[1][1]) <= tie_threshold:
                 # Create combined label, sorted alphabetically
                 combined_label = "+".join(sorted([top_two[0][0], top_two[1][0]]))
                 all_soz_labels_comb[subject_id_int] = combined_label
            else: # Clear dominant category
                all_soz_labels_comb[subject_id_int] = top_two[0][0]
        else: # No SOZ electrodes
             all_soz_labels_comb[subject_id_int] = 'Unknown' # Or 'None'

    # --- Process Resected (apply similar logic) ---
    res_df = electrodes_df[electrodes_df['resected'] == 'yes']
    if not res_df.empty:
        coarse_labels_res = [coarse_map.get(label, 'Other') for label in res_df['Destrieux_label_text']]

        if not coarse_labels_res:
             all_res_labels_comb[subject_id_int] = 'Unknown'
             continue

        count_res = Counter(coarse_labels_res)
        top_two_res = count_res.most_common(2)

        if len(top_two_res) == 1:
            all_res_labels_comb[subject_id_int] = top_two_res[0][0]
        elif len(top_two_res) == 2:
            if abs(top_two_res[0][1] - top_two_res[1][1]) <= tie_threshold:
                 combined_label_res = "+".join(sorted([top_two_res[0][0], top_two_res[1][0]]))
                 all_res_labels_comb[subject_id_int] = combined_label_res
            else:
                all_res_labels_comb[subject_id_int] = top_two_res[0][0]
        else:
            all_res_labels_comb[subject_id_int] = 'Unknown' # Or 'None'


print("\nLabels using Combined Labels for Ties/Close Counts:")
print(len(all_soz_labels_comb), all_soz_labels_comb)
print(len(all_res_labels_comb), all_res_labels_comb)

with open('.\\soz_cat_labels_combined.pkl', 'wb') as file:
    pickle.dump(all_soz_labels_comb, file)
with open('.\\soz_cat_labels_combined.pkl', 'wb') as file:
    pickle.dump(all_res_labels_comb, file)

sub-ccepAgeUMCU01 ['G:\\datasets\\sub-ccepAgeUMCU01\\ses-1\\ieeg\\sub-ccepAgeUMCU01_ses-1_electrodes.tsv']
sub-ccepAgeUMCU02 ['G:\\datasets\\sub-ccepAgeUMCU02\\ses-1\\ieeg\\sub-ccepAgeUMCU02_ses-1_electrodes.tsv']
sub-ccepAgeUMCU03 ['G:\\datasets\\sub-ccepAgeUMCU03\\ses-1\\ieeg\\sub-ccepAgeUMCU03_ses-1_electrodes.tsv']
sub-ccepAgeUMCU04 ['G:\\datasets\\sub-ccepAgeUMCU04\\ses-1\\ieeg\\sub-ccepAgeUMCU04_ses-1_electrodes.tsv']
sub-ccepAgeUMCU05 ['G:\\datasets\\sub-ccepAgeUMCU05\\ses-1\\ieeg\\sub-ccepAgeUMCU05_ses-1_electrodes.tsv']
sub-ccepAgeUMCU06 ['G:\\datasets\\sub-ccepAgeUMCU06\\ses-1\\ieeg\\sub-ccepAgeUMCU06_ses-1_electrodes.tsv']
sub-ccepAgeUMCU07 ['G:\\datasets\\sub-ccepAgeUMCU07\\ses-1\\ieeg\\sub-ccepAgeUMCU07_ses-1_electrodes.tsv']
sub-ccepAgeUMCU08 ['G:\\datasets\\sub-ccepAgeUMCU08\\ses-1\\ieeg\\sub-ccepAgeUMCU08_ses-1_electrodes.tsv']
sub-ccepAgeUMCU09 ['G:\\datasets\\sub-ccepAgeUMCU09\\ses-1b\\ieeg\\sub-ccepAgeUMCU09_ses-1b_electrodes.tsv']
sub-ccepAgeUMCU10 ['G:\\datasets\\s